In [ ]:
using OnePiece

## make_temporary

In [ ]:
te = OnePiece.Path.make_temporary("OnePiece.test.Path")

## clean

In [ ]:
OnePiece.Path.clean("a_b.c-d+e!f%g%h]iJK")

## error_extension

In [ ]:
fi = "file.extension"

;

In [ ]:
OnePiece.Path.error_extension(fi, ".extension")

In [ ]:
for ex in ("extension", ".another_extension")

    try

        OnePiece.Path.error_extension(fi, ex)

    catch er

        println(er)

    end

end

## replace_extension

In [ ]:
OnePiece.Path.replace_extension(fi, "new_extension")

## error_missing

In [ ]:
di = dirname(@__DIR__)

pr = "test/path.ipynb"

;

In [ ]:
OnePiece.Path.error_missing(di, (pr,))

In [ ]:
try

    OnePiece.Path.error_missing(di, (pr, "missing/file", "missing/directory/"))

catch er

    println(er)

end

## make_absolute

In [ ]:
for pa in ("~/file", "~/directory/")

    println(OnePiece.Path.make_absolute(pa))

end

## shorten

In [ ]:
pa = @__DIR__

In [ ]:
try

    OnePiece.Path.shorten(pa, "Shanks")

catch er

    println(er)

end

In [ ]:
for n in (0, 1, 2, count(ch -> ch == '/', pa) - 2, 9)

    println(OnePiece.Path.shorten(pa, n))

end

In [ ]:
for di in ("OnePiece.jl", "test", "OnePiece.jl/test")

    println(OnePiece.Path.shorten(pa, di))

end

In [ ]:
for sh in (0, -1, 1)

    println(OnePiece.Path.shorten(pa, dirname(dirname(pa)), sh))

end

## select

In [ ]:
di = homedir()

In [ ]:
OnePiece.Path.select(di)

In [ ]:
OnePiece.Path.select(di, false, ig_ = (), ke_ = (r"^\.",))

## move

In [ ]:
di1 = mkpath(joinpath(te, "di1"))

di2 = mkpath(joinpath(te, "di2"))

di3 = mkpath(joinpath(te, "di3"))

fi1 = joinpath(di1, "fi1")

fi2 = joinpath(di2, "fi2")

touch(fi1)

touch(fi2)

run(`tree $te`)

In [ ]:
fi3 = replace(fi1, "di1" => "di3")

OnePiece.Path.move(fi1, fi3)

In [ ]:
run(`tree $te`)

In [ ]:
try

    OnePiece.Path.move(di2, di3)

catch er

    println(er)

end

In [ ]:
println(OnePiece.Path.move(di2, di3, force = true))

run(`tree $te`)

## sed_recursively

In [ ]:
fi1 = joinpath(te, "fi1")

fi2 = joinpath(te, "fi2")

open(fi1, "w") do io

    write(io, "Before")

end

open(fi2, "w") do io

    write(io, "BeforeBefore")

end

println(readline(open(fi1)))

println(readline(open(fi2)))

In [ ]:
OnePiece.Path.sed_recursively(te, ("Before" => "After",))

println(readline(open(fi1)))

println(readline(open(fi2)))

## rename_recursively

In [ ]:
fi1 = joinpath(te, "fi1")

fi2 = joinpath(te, "fi2")

touch(fi1)

touch(fi2)

readdir(te)

In [ ]:
OnePiece.Path.rename_recursively(te, ("fi" => "new",))

readdir(te)